In [259]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors ar|e:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculat the Mean Squared Error and use it as loss.
    mse = mean(square(y_true_slice - y_pred_slice))
    
    return mse

def init_model():
    model = Sequential()

    model.add(GRU(units=512,
                  return_sequences=True,
                  input_shape=(None, num_x_signals,)))

    model.add(Dense(num_y_signals, activation='sigmoid'))

    optimizer = RMSprop(lr=1e-3)

    model.compile(loss=loss_mse_warmup, optimizer=optimizer)

    model.summary()

    return model

def pred(model, pred_x_data, y_true):
    '''
    수익률 계산을 위한 실제 주식값, 예측 주식값 생성
    '''
#     pred_x_data = df.values[shift_steps:-1]
#     y_true = np.array(df_targets.values[shift_steps:-1], dtype=np.int)# .reshape(-1,1)

    #print(y_true[-10:])

    pred_x_train_scaled = x_scaler.fit_transform(pred_x_data)
    # y_true_scaled = y_scaler.transform(y_true)

    pred_x = np.expand_dims(pred_x_train_scaled, axis=0)

    pred = model.predict(pred_x)

    pred_rescaled = y_scaler.inverse_transform(pred[0])

    pred = pred_rescaled[:, 0].astype(np.int)

    return y_true, pred

In [260]:
%matplotlib inline
import pymysql
import datetime as dt
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
from dateutil.parser import parse

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

In [261]:
print(tf.__version__)

print(tf.keras.__version__)

print(pd.__version__)
# GPU 확인
tf.test.is_gpu_available( cuda_only=False, min_cuda_compute_capability=None )

2.0.0
2.2.4-tf
1.0.1


True

In [262]:
data = pd.read_csv('result.csv', encoding='cp949').set_index('date')
print(data.shape)
data.head(2)

(1501, 36)


,아시아종묘,조비,효성오앤비,경농,남해화학,KG케미칼,농우바이오,성보화학,아세아텍,동방아그로,...,green_onion,cucumber,price_egg,price_milk,price_sugar,exchangerate,kospi,Dubai,Brent,WTI
date,,,,,,,,,,,,,,,,,,,,,
2014-08-01,3300.0,12000.0,17300.0,5780.0,9260.0,13700.0,20650.0,42350.0,3455.0,6380.0,...,1156.0,62722.0,5987.0,2549.0,435.7,1037.75,2073.10,104.92,104.84,97.88
2014-08-04,3300.0,12000.0,17350.0,5900.0,9280.0,13700.0,21100.0,43050.0,3465.0,6370.0,...,1510.0,43128.0,5987.0,2549.0,468.9,1033.35,2080.42,103.15,105.41,98.29


In [263]:
data.loc[str(parse(data.index[-1]).date() + timedelta(days=1))] = [np.nan for _ in range(data.shape[1])]

data.tail(2)

,아시아종묘,조비,효성오앤비,경농,남해화학,KG케미칼,농우바이오,성보화학,아세아텍,동방아그로,...,green_onion,cucumber,price_egg,price_milk,price_sugar,exchangerate,kospi,Dubai,Brent,WTI
date,,,,,,,,,,,,,,,,,,,,,
2020-03-31,3750.0,12350.0,5400.0,9580.0,6990.0,10850.0,8800.0,3300.0,3590.0,5400.0,...,844.0,43002.0,5311.0,2623.0,330.9,1218.8,1754.64,23.43,22.74,20.48
2020-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
target_name = 'KPX생명과학'
target_names = [target_name,'cabbage1']
#df = data[]
shift_steps = 1

df = data[target_names].shift(1)

df.tail(3)

,KPX생명과학,cabbage1
date,,
2020-03-30,4750.0,11277.0
2020-03-31,5030.0,13479.0
2020-04-01,5300.0,12311.0


In [265]:
df_targets = df[target_name].shift(-shift_steps)
# df_targets = df.shift(-shift_steps)

df_targets.tail(3)

date
2020-03-30    5030.0
2020-03-31    5300.0
2020-04-01       NaN
Name: KPX생명과학, dtype: float64

In [266]:
x_data = df.values[shift_steps:-23] # 3월 예측

print(type(x_data))
print("Shape:", x_data.shape)
y_data = df_targets.values[shift_steps:-23].reshape(-1,1)

print(type(y_data))
print("Shape:", y_data.shape)

<class 'numpy.ndarray'>
Shape: (1478, 2)
<class 'numpy.ndarray'>
Shape: (1478, 1)


In [267]:
num_data = len(x_data)
print(num_data)
train_split = 0.9

num_train = int(train_split * num_data)
print(num_train)

num_test = num_data - num_train
print(num_test)

x_train = x_data[0:num_train]
x_test = x_data[num_train:]
print(len(x_train) + len(x_test))

y_train = y_data[0:num_train]
y_test = y_data[num_train:]
print(len(y_train) + len(y_test))

1478
1330
148
1478
1478


In [268]:
# input 갯수
num_x_signals = x_data.shape[1]
# output 갯수
num_y_signals = y_data.shape[1]

print('input ouput 갯수 :',num_x_signals, num_y_signals)

input ouput 갯수 : 2 1


In [269]:
# Scaled Data
x_scaler = MinMaxScaler()

x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
# print("Min:", np.min(x_train_scaled))
# print("Max:", np.max(x_train_scaled))

In [270]:
batch_size = 256
sequence_length = 365
warmup_steps = 100

generator = batch_generator(batch_size=batch_size, sequence_length=sequence_length)
x_batch, y_batch = next(generator)

print(x_batch.shape)
print(y_batch.shape)

(256, 365, 2)
(256, 365, 1)


### Model 불러오기

In [271]:
model = init_model()
# [_stock_Jobi_, _onion_].h5
model.load_weights('model/[\'KPX생명과학\', \'cabbage1\'].h5')# Re-evaluate the model , \'price_egg\'

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_11 (GRU)                 (None, None, 512)         792576    
_________________________________________________________________
dense_11 (Dense)             (None, None, 1)           513       
Total params: 793,089
Trainable params: 793,089
Non-trainable params: 0
_________________________________________________________________


In [272]:
t,p = pred(model, df.values[shift_steps:], np.array(df_targets.values[shift_steps:], dtype=np.float))

In [295]:
# date
date = np.array(df.reset_index()['date'][:-1]).reshape(-1,1)

# com_name
com_name = np.array([target_name for _ in range(len(date))]).reshape(-1,1)

# com_code
com_code = np.array(['114450' for _ in range(len(date))]).reshape(-1,1)

# tod_price
tod_price = np.array(data[target_name][:-1]).reshape(-1,1)


# tod_status
tod_status = np.array([0 for _ in range(len(date))]).reshape(-1,1)
sub = tod_price[1:] - tod_price[:-1]
for idx, value in enumerate(sub):
    if value > 0: tod_status[idx+1] = 1
    elif value < 0: tod_status[idx+1] = -1

# tom_price
tom_price = np.array(data[target_name][:-1])
tom_price[1:] += p[1:] - p[:-1]
tom_price = tom_price.reshape(-1,1)

# tom_status
# 오늘 종가로부터 상향, 하향, 유지
tom_status = np.array([0 for _ in range(len(date))]).reshape(-1,1)
for idx, value in enumerate(tom_price - tod_price):
    if value > 0: tom_status[idx] = 1
    elif value < 0: tom_status[idx] = -1
        
# match_status
# 작일 예측 여부 확인
match_status = np.array([True for _ in range(len(date))]).reshape(-1,1)
for idx, value in enumerate(tom_status[:-1] == tod_status[1:]):
    match_status[idx+1] = value

# price_error
# 작일 예측값 - 금일 종가의 절댓값
price_error = np.array([0 for _ in range(len(date))]).reshape(-1,1)
for idx, value in enumerate(tom_price[:-1] - tod_price[1:]):
    price_error[idx + 1] = abs(value)
    
# return
# 금일 수익률
# 작일 tom_status > 0 => (금일 tod_price) - (작일 tod_price) 만큼 수익 발생
# 작일 tom_status <= 0 => 수익 없음
returns = np.array([1.0 for _ in range(len(date))]).reshape(-1,1)

for idx, value in enumerate(tom_status[:-1]):
    if value == 1:
        returns[idx] += (tod_price[idx+1] - tod_price[idx]) / tod_price[idx]
returns = np.round(returns,3)


nan = np.array([np.nan for _ in range(len(date))]).reshape(-1,1)

In [296]:
returns.shape

(1501, 1)

In [297]:
DB_data = pd.DataFrame(np.concatenate([com_name,com_code,date,tod_price,tod_status,tom_price,tom_status,match_status,price_error,returns], axis = 1),columns = ['com_name','com_code','date','tod_price','tod_status','tom_price','tom_status','match_status','price_error','return'])
# com_name,date,tod_price,tod_status,tom_price,tom_status,match_status,price_error,returns
DB_data

,com_name,com_code,date,tod_price,tod_status,tom_price,tom_status,match_status,price_error,return
0,KPX생명과학,114450,2014-08-01,3020,0,3020,0,True,0,1
1,KPX생명과학,114450,2014-08-04,3050,1,2600,-1,False,30,1
2,KPX생명과학,114450,2014-08-05,3250,1,3282,1,False,650,0.974
3,KPX생명과학,114450,2014-08-06,3165,-1,3121,-1,False,117,1
4,KPX생명과학,114450,2014-08-07,3160,-1,3088,-1,True,39,1
...,...,...,...,...,...,...,...,...,...,...
1496,KPX생명과학,114450,2020-03-25,4190,1,4417,1,True,288,1.041
1497,KPX생명과학,114450,2020-03-26,4360,1,4501,1,True,57,1.089
1498,KPX생명과학,114450,2020-03-27,4750,1,5029,1,True,249,1.059
1499,KPX생명과학,114450,2020-03-30,5030,1,5316,1,True,1,1.054


In [298]:
# 2020-01-01 이후 수익률
np.prod(returns[-65:-1])

1.3772701152830806

In [299]:
# returns_lst = pd.DataFrame(columns = ['name','error','returns','stock_name']).set_index('name')
returns_lst = pd.read_csv('returns_lst.csv').set_index('name')

returns_lst.head()

,error,returns,stock_name
name,,,
"['아세아텍', 'price_egg']",0.00213,1.570,아세아텍
"['효성오앤비', 'kospi']",0.00122,1.492,효성오앤비
"['아세아텍', 'exchangerate']",0.00223,1.473,아세아텍
"['남해화학', 'price_egg']",0.00088,1.468,남해화학
"['남해화학', 'price_sugar']",0.00113,1.468,남해화학


In [300]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))

print("loss (test-set):", result)

returns_lst.loc[str([str(i) for i in df.columns])] = [round(result,5), round(np.prod(returns[-65:-1]),3), target_name]
# result_lst.append([str([str(i) for i in df.columns]), (round(result,5))])
returns_lst[returns_lst['stock_name'] == target_name].sort_values(by = ['returns','error'], ascending=[False, True])

1/1 [==============================] - 0s 26ms/sample - loss: 0.0329
loss (test-set): 0.03292461112141609


,error,returns,stock_name
name,,,
"['KPX생명과학', 'cabbage1']",0.03292,1.377,KPX생명과학
['KPX생명과학'],0.00999,1.307,KPX생명과학
"['KPX생명과학', 'cabbage']",0.01073,1.293,KPX생명과학
"['KPX생명과학', 'tomato']",0.00612,1.207,KPX생명과학
"['KPX생명과학', 'kospi']",0.00520,1.205,KPX생명과학
"['KPX생명과학', 'price_egg']",0.01118,1.183,KPX생명과학
"['KPX생명과학', 'exchangerate']",0.00750,1.112,KPX생명과학
"['KPX생명과학', 'potato']",0.00443,1.109,KPX생명과학


In [301]:
result = returns_lst.sort_values(by = ['returns','error'], ascending=[False, True])#.set_index('name')
result.to_csv('returns_lst.csv')
result.head()

,error,returns,stock_name
name,,,
"['아세아텍', 'price_egg']",0.00213,1.570,아세아텍
"['효성오앤비', 'kospi']",0.00122,1.492,효성오앤비
"['아세아텍', 'exchangerate']",0.00223,1.473,아세아텍
"['남해화학', 'price_egg']",0.00088,1.468,남해화학
"['남해화학', 'price_sugar']",0.00113,1.468,남해화학


In [302]:
result['stock_name'].unique()

array(['아세아텍', '효성오앤비', '남해화학', '조비', '경농', 'KPX생명과학', 'KG케미칼', '아시아종묘',
       '농우바이오', '동방아그로'], dtype=object)

In [303]:
result[result['stock_name'] == 'KPX생명과학']

,error,returns,stock_name
name,,,
"['KPX생명과학', 'cabbage1']",0.03292,1.377,KPX생명과학
['KPX생명과학'],0.00999,1.307,KPX생명과학
"['KPX생명과학', 'cabbage']",0.01073,1.293,KPX생명과학
"['KPX생명과학', 'tomato']",0.00612,1.207,KPX생명과학
"['KPX생명과학', 'kospi']",0.00520,1.205,KPX생명과학
"['KPX생명과학', 'price_egg']",0.01118,1.183,KPX생명과학
"['KPX생명과학', 'exchangerate']",0.00750,1.112,KPX생명과학
"['KPX생명과학', 'potato']",0.00443,1.109,KPX생명과학


In [304]:
DB_data.match_status[-50:].mean()

0.6

In [305]:
data.columns

Index(['아시아종묘', '조비', '효성오앤비', '경농', '남해화학', 'KG케미칼', '농우바이오', '성보화학', '아세아텍',
       '동방아그로', 'KPX생명과학', 'SPC삼립', '풀무원', '농심', '오뚜기', '오리온', '해태제과', '롯데제과',
       'potato', 'tomato', 'green_pepper', 'red_pepper', 'cabbage', 'cabbage1',
       'onion', 'carrot', 'green_onion', 'cucumber', 'price_egg', 'price_milk',
       'price_sugar', 'exchangerate', 'kospi', 'Dubai', 'Brent', 'WTI'],
      dtype='object')

In [306]:
def cor_to_database(data):
    conn = pymysql.connect(host='192.168.1.23', user='root', password='1231',
                               db='bms_test', charset='utf8')

    curs = conn.cursor()
    sql = '''INSERT INTO stock_corr VALUES (%s,%s,%s)'''
    
    data = data.corr().loc[:'롯데제과', :'롯데제과']
    for i in data.index:
        for j in data.columns:
            if data[i][j] != 1:
                curs.execute(sql, tuple([i, j, float(data[i][j])]))
                # print(tuple([i, j, data[i][j]]))
    conn.commit()
    
# cor_to_database(data)

### 주가 코드 변경

In [307]:
def db_to_database(DB_data):
    conn = pymysql.connect(host='192.168.1.23', user='root', password='1231',
                               db='bms_test', charset='utf8')

    curs = conn.cursor()

    sql = '''INSERT INTO stock_predict VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

    for data in DB_data.values:
        curs.execute(sql, (tuple(data)))

    conn.commit()

db_to_database(DB_data)